In [1]:
import pandas as pd
from openai import OpenAI
from utils.keyword_extractor import keyword_document_mapping

In [2]:
data=pd.read_csv("data/pubchem_dump_with_wiki_text.csv")

In [8]:
print(data.iloc[137]['combined_text'])

wikipedia: 
 pubchem:
5-aminopentanoic acid is a delta-amino acid comprising pentanoic acid with an amino substituent at C-5; a methylene homologue of gamma-aminobutyric acid (GABA) that is a weak GABA agonist. It has a role as a human metabolite. It is a delta-amino acid and an omega-amino fatty acid. It is functionally related to a valeric acid. It is a conjugate acid of a 5-aminopentanoate. It is a tautomer of a 5-aminopentanoic acid zwitterion.
5-Aminopentanoate has been reported in Daphnia pulex, Homo sapiens, and other organisms with data available.
safety: Irritant
smiles: C(CCN)CC(=O)O
forumla: C5H11NO2
chem_properties: Molecular Weight: 117.15 g/mol
XLogP3: -2.6
Hydrogen Bond Donor Count: 2
Hydrogen Bond Acceptor Count: 3
Rotatable Bond Count: 4
Exact Mass: 117.078978594 Da
Monoisotopic Mass: 117.078978594 Da
Topological Polar Surface Area: 63.3
Heavy Atom Count: 8
Formal Charge: 0
Complexity: 72.8
Isotope Atom Count: 0
Defined Atom Stereocenter Count: 0
Undefined Atom Stereoc

In [11]:
data['text'].fillna('')+'test'

0        \nO-acetylcarnitine is an O-acylcarnitine havi...
1                                                     test
2        \n2,3-dihydroxy-2,3-dihydrobenzoic acid is a c...
3        \nMonoisopropanolamine appears as a colorless ...
4        \n3-amino-2-oxopropyl phosphate is a oxoalkyl ...
                               ...                        
49995    \n2,2',4,5,5'-Pentabromobiphenyl is a polybrom...
49996                                                 test
49997                                                 test
49998    \nHomobaldrinal is a fatty acid ester.\nHomoba...
49999                                                 test
Name: text, Length: 50000, dtype: object

In [9]:
data['wiki_text']+pd.Series(['test' for _ in range(len(data))])

0                                                      NaN
1                                                      NaN
2                                                      NaN
3        1-Aminopropan-2-ol is the organic compound wit...
4                                                      NaN
                               ...                        
49995                                                  NaN
49996                                                  NaN
49997                                                  NaN
49998                                                  NaN
49999                                                  NaN
Length: 50000, dtype: object

In [14]:
data=data.dropna(subset=['name','combined_text'])

In [18]:
data

,Unnamed: 0,text,mentioned_entity,cid,name,properties,wiki_text,combined_text
0,0,\nO-acetylcarnitine is an O-acylcarnitine havi...,"[""7045767"", ""176"", ""288"", ""5962"", ""6137""]",1,Acetyl-DL-carnitine,safety: \nsmiles: CC(=O)OC(CC(=O)[O-])C[N+](C)...,NaN,wikipedia: \n pubchem:\nO-acetylcarnitine is a...
1,1,NaN,[],2,[2-(Acetyloxy)-3-carboxypropyl]trimethylazanium,safety: \nsmiles: CC(=O)OC(CC(=O)O)C[N+](C)(C)...,NaN,NaN
2,2,"\n2,3-dihydroxy-2,3-dihydrobenzoic acid is a c...","[""961"", ""23615184""]",3,"5,6-Dihydroxycyclohexa-1,3-diene-1-carboxylic ...",safety: \nsmiles: C1=CC(C(C(=C1)C(=O)O)O)O\nfo...,NaN,"wikipedia: \n pubchem:\n2,3-dihydroxy-2,3-dihy..."
3,3,\nMonoisopropanolamine appears as a colorless ...,"[""962"", ""222"", ""787"", ""15805271""]",4,1-Aminopropan-2-ol,safety: Corrosive\nsmiles: CC(CN)O\nforumla: C...,1-Aminopropan-2-ol is the organic compound wit...,wikipedia: 1-Aminopropan-2-ol is the organic c...
4,4,\n3-amino-2-oxopropyl phosphate is a oxoalkyl ...,[],5,3-Amino-2-oxopropyl phosphate,safety: \nsmiles: C(C(=O)COP(=O)(O)O)N\nforuml...,NaN,wikipedia: \n pubchem:\n3-amino-2-oxopropyl ph...
...,...,...,...,...,...,...,...,...
49995,49995,"\n2,2',4,5,5'-Pentabromobiphenyl is a polybrom...","[""7095""]",49996,"2,2',4,5,5'-Pentabromobiphenyl",safety: \nsmiles: C1=CC(=C(C=C1Br)C2=CC(=C(C=C...,NaN,"wikipedia: \n pubchem:\n2,2',4,5,5'-Pentabromo..."
49996,49996,NaN,[],49997,"Cyclopropanecarboxylic acid, 2,2,3,3-tetrameth...",safety: \nsmiles: CC1(C(C1(C)C)C(=O)O[C@H](C#N...,NaN,NaN
49997,49997,NaN,[],49998,"1-Phenanthrenecarboxylic acid, 1,2,3,4,4a,4b,5...",safety: \nsmiles: CC(C)C1CCC2C(=CCC3C2(CCCC3(C...,NaN,NaN
49998,49998,\nHomobaldrinal is a fatty acid ester.\nHomoba...,[],49999,Homobaldrinal,safety: \nsmiles: CC(C)CC(=O)OCC1=COC=C2C1=CC=...,NaN,wikipedia: \n pubchem:\nHomobaldrinal is a fat...


In [15]:
keyword_document_mapping(list(data.combined_text),list(data.name))

317it [00:46,  6.75it/s]


KeyboardInterrupt: 

In [4]:
data.name

,Unnamed: 0,text,mentioned_entity,cid,name,properties,wiki_text,combined_text
0,0,\nO-acetylcarnitine is an O-acylcarnitine havi...,"[""7045767"", ""176"", ""288"", ""5962"", ""6137""]",1,Acetyl-DL-carnitine,safety: \nsmiles: CC(=O)OC(CC(=O)[O-])C[N+](C)...,NaN,wikipedia: \n pubchem:\nO-acetylcarnitine is a...
1,1,NaN,[],2,[2-(Acetyloxy)-3-carboxypropyl]trimethylazanium,safety: \nsmiles: CC(=O)OC(CC(=O)O)C[N+](C)(C)...,NaN,NaN
2,2,"\n2,3-dihydroxy-2,3-dihydrobenzoic acid is a c...","[""961"", ""23615184""]",3,"5,6-Dihydroxycyclohexa-1,3-diene-1-carboxylic ...",safety: \nsmiles: C1=CC(C(C(=C1)C(=O)O)O)O\nfo...,NaN,"wikipedia: \n pubchem:\n2,3-dihydroxy-2,3-dihy..."
3,3,\nMonoisopropanolamine appears as a colorless ...,"[""962"", ""222"", ""787"", ""15805271""]",4,1-Aminopropan-2-ol,safety: Corrosive\nsmiles: CC(CN)O\nforumla: C...,1-Aminopropan-2-ol is the organic compound wit...,wikipedia: 1-Aminopropan-2-ol is the organic c...
4,4,\n3-amino-2-oxopropyl phosphate is a oxoalkyl ...,[],5,3-Amino-2-oxopropyl phosphate,safety: \nsmiles: C(C(=O)COP(=O)(O)O)N\nforuml...,NaN,wikipedia: \n pubchem:\n3-amino-2-oxopropyl ph...
...,...,...,...,...,...,...,...,...
49995,49995,"\n2,2',4,5,5'-Pentabromobiphenyl is a polybrom...","[""7095""]",49996,"2,2',4,5,5'-Pentabromobiphenyl",safety: \nsmiles: C1=CC(=C(C=C1Br)C2=CC(=C(C=C...,NaN,"wikipedia: \n pubchem:\n2,2',4,5,5'-Pentabromo..."
49996,49996,NaN,[],49997,"Cyclopropanecarboxylic acid, 2,2,3,3-tetrameth...",safety: \nsmiles: CC1(C(C1(C)C)C(=O)O[C@H](C#N...,NaN,NaN
49997,49997,NaN,[],49998,"1-Phenanthrenecarboxylic acid, 1,2,3,4,4a,4b,5...",safety: \nsmiles: CC(C)C1CCC2C(=CCC3C2(CCCC3(C...,NaN,NaN
49998,49998,\nHomobaldrinal is a fatty acid ester.\nHomoba...,[],49999,Homobaldrinal,safety: \nsmiles: CC(C)CC(=O)OCC1=COC=C2C1=CC=...,NaN,wikipedia: \n pubchem:\nHomobaldrinal is a fat...


In [11]:
multi_hop=[]
row=data[data.mentioned_entity!='[]'].iloc[0]
mentioned_entity=[int(item)-1 for item in eval(row['mentioned_entity']) if int(item)<=50000]
for edge in mentioned_entity:
    next_row=data.iloc[edge]
    if len(next_row['combined_text'])>10:
        qa={'source':row['name'],'bridge':next_row['name'],'text1':row['combined_text'],'text2':next_row['combined_text'],}
        multi_hop.append(qa)

In [20]:
import dotenv
import os
dotenv.load_dotenv()
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
prompt = (
    f"You are given two texts and a bridge word to connect them.\n\n"
    f"Text1: {multi_hop[0]['text1']}\n"
    f"Bridge Word: {multi_hop[0]['bridge']}\n"
    f"Text2: {multi_hop[0]['text2']}\n\n"
    f"Your job is to generate a multihop question that requires the user to read the first document find the bridge word and using the bridge word go to the second document. "
    f"You should not mention the bridge word directly so it requires the person to read the first document first"
    f"the question should be a single one with a closed brief answer"
    f"Return a dictionary without any code formatting, backticks, or markdown , with keys 'q' and 'a' "
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="gpt-4o",
)
print (response.choices[0].message.content.strip())

{ 
  "q": "In which countries is the investigational drug, synthesized from an amino acid derivative that is made in the kidney, liver, and brain from lysine and methionine, approved for use, and what notable role does its carboxylic acid precursor play in household and food industries?", 
  "a": "The investigational drug is approved in Italy, Portugal, Argentina, Chile, Philippines, Australia, and India. Its carboxylic acid precursor plays an important role as a descaling agent in households and as an acidity regulator in the food industry." 
}


In [40]:
import dotenv
import os
dotenv.load_dotenv()
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
prompt = (
    f"You are given a text and a bridge word.\n\n"
    f"Text: {multi_hop[0]['text1']}\n"
    f"Bridge Word: {multi_hop[0]['bridge']}\n"
    f"Your job is to generate a question that requires the user to read the  document and find the bridge word as the answer. "
    f"Return a dictionary without any code formatting, backticks, or markdown , with keys 'q' and 'a' "
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="gpt-4o",
)
q1= eval(response.choices[0].message.content.strip())

In [41]:
import dotenv
import os
dotenv.load_dotenv()
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
prompt = (
    f"You are given a text and a bridge word.\n\n"
    f"Text: {multi_hop[0]['text2']}\n"
    f"Key Word: {multi_hop[0]['bridge']}\n"
    f"Your job is to generate a question about the Key word "
    f"Return a dictionary without any code formatting, backticks, or markdown , with keys 'q' and 'a' "
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="gpt-4o",
)
q2= eval(response.choices[0].message.content.strip())

In [42]:
import dotenv
import os
dotenv.load_dotenv()
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
prompt = (
    f"You are given two question with their answers and a bridge word.\n\n"
    f"question1: {q1['q']}\n"
    f"answer1: {q1['a']}\n"
    f"question2: {q2['q']}\n"
    f"answer2: {q2['a']}\n"
    f"Your job is to generate a multi-hop question, that combines these two questions.  "
    f"The question should require the person to find the answer to the first question then look for the answer of the second question"
    f"Return a dictionary without any code formatting, backticks, or markdown , with keys 'q' and 'a' "
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="gpt-4o",
)
print(response.choices[0].message.content.strip())

{"q": "What is the primary industrial method for the production of the compound that O-acetylcarnitine is functionally related to, and what are the main catalysts used in this process?", "a": "The primary industrial method for the production of acetic acid, which O-acetylcarnitine is functionally related to, is the carbonylation of methanol. The main catalysts used in this process are rhodium-based catalysts for the Monsanto process and iridium-based catalysts for the Cativa process."}


In [38]:
prompt

"You are given two question with their answers and a bridge word.\n\nquestion1: What compound is O-acetylcarnitine functionally related to?\nanswer1: Acetic Acid\nquestion2: What is the primary industrial method used to produce acetic acid worldwide?\nanswer2: The primary industrial method used to produce acetic acid worldwide is the carbonylation of methanol, specifically through the Cativa process, which uses an iridium-catalyzed reaction to produce acetic acid more efficiently.\nYour job is to generate a multi-hop question, that combines these two questions.  The question should require the person to find the answer to the first question then look for the answer of the second questionReturn a dictionary without any code formatting, backticks, or markdown , with keys 'q' and 'a' "